In [1]:
import functools
import json
import math
import numpy as np
import pandas as pd
import requests
import time

In [2]:
test_base_url = 'https://sandbox.iexapis.com'
test_token = 'Tpk_27c608f1ca1a41a1984f49517f378203'

base_url = 'https://cloud.iexapis.com'
token = 'pk_8d9bd95ebeaf42f080b79d8383279409'

In [3]:
# get all the symbols and store in df
symbol_url = '{}/v1/ref-data/symbols?token={}'.format(test_base_url, test_token)
try:
    r = requests.get(symbol_url)
    print(r)
except Exception as e:
    print(e)

print(r.headers['iexcloud-messages-used'])

<Response [200]>
100


In [4]:
symbols_df = pd.read_json(json.dumps(r.json()))
print(len(symbols_df))
filtered_symbols_df = symbols_df[(symbols_df.type == 'cs')]
print(len(filtered_symbols_df))

# data = pd.DataFrame()
# print(filtered_symbols_df.shape)
print(filtered_symbols_df.head())
# set_index=True

8814
4565
  currency       date exchange                 iexId  isEnabled  \
0      USD 2019-07-27      SYN  IEX_46574843354B2D52       True   
1      USD 2019-07-27      YSN  IEX_4238333734532D52       True   
4      USD 2019-07-27      SNY  IEX_4843364642592D52       True   
6      USD 2019-07-27      ANS  IEX_4353464A535A2D52       True   
7      USD 2019-07-27      ASE  IEX_5442323844432D52       True   

                                name region symbol type  
0          neinlIlsgo. ce hgciTotnAe     US      A   cs  
1                        .op orAcCal     US     AA   cs  
4                  AnHig n.lsCAcId o     US    AAC   cs  
6       clG.aierAnue crniAnsior  mpI     US    AAL   cs  
7  Mrtoosimapseeen AutgaC c.ntrs lAe     US   AAMC   cs  


In [5]:
# create batches
# symbols max = 100
# '{}/v1/stock/market/batch?symbols={}&types=chartt&range=5y&token={}'.format()
symbol_chunk_list = []

# create chunks from filtered_symbols_df
chunks = math.ceil(len(filtered_symbols_df)/100)
for chunk in np.array_split(filtered_symbols_df, chunks):
    symbol_chunk = ','.join(chunk['symbol'].tolist())
    symbol_chunk_list.append(symbol_chunk)
    
print(len(symbol_chunk_list))
print(symbol_chunk_list[0])

46
A,AA,AAC,AAL,AAMC,AAME,AAN,AAOI,AAON,AAP,AAPL,AAT,AAU,AAWW,AAXN,AB,ABBV,ABC,ABCB,ABDC,ABEO,ABG,ABIL,ABIO,ABM,ABMD,ABR,ABT,ABTX,ABUS,AC,ACA,ACAD,ACAM,ACB,ACBI,ACC,ACCO,ACER,ACGL,ACHC,ACHN,ACHV,ACIA,ACIU,ACIW,ACLS,ACM,ACMR,ACN,ACNB,ACOR,ACRE,ACRS,ACRX,ACST,ACTG,ACTT,ACU,ACY,ADBE,ADC,ADES,ADI,ADIL,ADM,ADMA,ADMP,ADMS,ADNT,ADOM,ADP,ADPT,ADRO,ADS,ADSK,ADSW,ADT,ADTN,ADUS,ADVM,ADXS,AE,AEE,AEGN,AEHR,AEIS,AEL,AEM,AEMD,AEO,AEP,AER,AERI,AES,AETI,AEY,AEYE,AEZS,AFG


In [6]:
data = pd.DataFrame()
set_index = True
iexcloud_msg_used = 0
call_count = 0

start = time.time()
for symbol_list in symbol_chunk_list:
    
    batch_url = '{}/v1/stock/market/batch?symbols={}&types=chart&range=5y&filter=date,close&token={}'.format(test_base_url, symbol_list, test_token)
    try:
        batch_request = requests.get(batch_url)
        iexcloud_msg_used = iexcloud_msg_used + int(batch_request.headers['iexcloud-messages-used'])
        print("msg used: {}".format(iexcloud_msg_used))
        call_count = call_count + 1
        print("call count: {}".format(call_count))
    except Exception as e:
        print(e)
    
    json_response = batch_request.json()
    
    for symbol in json_response:

        zipped_data = zip(
            [i['date'] for i in json_response[symbol]['chart']],
            [i['close'] for i in json_response[symbol]['chart']]
        )
        list_of_zipped_data = list(zipped_data)
        symbol_df = pd.DataFrame(list_of_zipped_data, columns = ['date', 'close'])
        
        if set_index:
            data['date'] = symbol_df['date']
            data[symbol] = symbol_df['close']
            # changes the index col to dtype str
            data.rename(index=str, columns={symbol: symbol}, inplace=True)
            data.set_index('date', inplace=True)
            # sets flag to false so this block isn't run anymore
            set_index = False
        else:
            symbol_df.rename(index=str, columns={'close': symbol}, inplace=True)
            symbol_df.set_index('date', inplace=True)
            data = data.join(symbol_df, on='date', how='left')

end = time.time()
print(end - start)

msg used: 1129750
call count: 1
msg used: 2163720
call count: 2
msg used: 3231250
call count: 3
msg used: 4310360
call count: 4
msg used: 5390980
call count: 5
msg used: 6510230
call count: 6
msg used: 7552190
call count: 7
msg used: 8650160
call count: 8
msg used: 9757030
call count: 9
msg used: 10914870
call count: 10
msg used: 12025860
call count: 11
msg used: 13144570
call count: 12
msg used: 14249030
call count: 13
msg used: 15354620
call count: 14
msg used: 16432420
call count: 15
msg used: 17587990
call count: 16
msg used: 18618590
call count: 17
msg used: 19694930
call count: 18
msg used: 20709010
call count: 19
msg used: 21790580
call count: 20
msg used: 22935740
call count: 21
msg used: 23988390
call count: 22
msg used: 25120910
call count: 23
msg used: 26165200
call count: 24
msg used: 27234190
call count: 25
msg used: 28334280
call count: 26
msg used: 29432400
call count: 27
msg used: 30525600
call count: 28
msg used: 31610440
call count: 29
msg used: 32736730
call count: 3

In [8]:
data.to_csv('/Users/joshnielsen/Desktop/historical_data.csv')

In [30]:
data.shape
data.head()
data['ZOM'][500:600]

date
2016-07-21       NaN
2016-07-22       NaN
2016-07-25       NaN
2016-07-26       NaN
2016-07-27       NaN
2016-07-28       NaN
2016-07-29    0.8406
2016-08-01    0.8513
2016-08-02    0.8588
2016-08-03    0.8645
2016-08-04    0.8378
2016-08-05    0.8686
2016-08-08    0.9340
2016-08-09    0.9414
2016-08-10    0.9739
2016-08-11    0.9960
2016-08-12    0.9853
2016-08-15    1.1400
2016-08-16    1.1300
2016-08-17    1.1200
2016-08-18    1.1700
2016-08-19    1.1400
2016-08-22    1.1600
2016-08-23    1.1000
2016-08-24    1.1100
2016-08-25    1.1400
2016-08-26       NaN
2016-08-29       NaN
2016-08-30    1.0000
2016-08-31       NaN
               ...  
2016-10-28    1.1300
2016-10-31    1.1700
2016-11-01    1.1600
2016-11-02    1.1700
2016-11-03    1.1700
2016-11-04    1.1600
2016-11-07    1.1400
2016-11-08    1.1800
2016-11-09    1.1000
2016-11-10    1.1000
2016-11-11    1.1000
2016-11-14    1.1000
2016-11-15    1.1200
2016-11-16    1.1100
2016-11-17    1.0400
2016-11-18    1.0800
2016-11-

In [ ]:
a = pd.DataFrame({'a':[1,2,3], 'b':[4,5,6]})
a = a.set_index('a')

In [ ]:
b = pd.DataFrame({'a':[2,3], 'c':[9,9]})
b = b.set_index('a')

In [ ]:
a = a.join(b, how='left')
a

In [ ]:
a.index.tolist()